<img src="https://www.mines.edu/webcentral/wp-content/uploads/sites/267/2019/02/horizontallightbackground.jpg" width="100%"> 

### CSCI250 Python Computing: Building a Sensor System
<hr style="height:5px" width="100%" align="left">

# Inter-Integrated Circuit interface

# Introduction

This notebook covers the basic principles of the **Inter-Integrated Circuit interface** (I<sup>2</sup>C) and how to use it on the RPi. The only I<sup>2</sup>C device that we will use in this class is the accelerometer, so this notebook uses it as an example; the process for using other I<sup>2</sup>C devices is similar.

This notebook applies specifically to the [**LIS3DH**](https://www.sparkfun.com/products/13963) accelerometer.

# I<sup>2</sup>C interface

The accelerometer communicates with the RPi over the protocol called [Inter-Integrated Circuit (I<sup>2</sup>C)](https://en.wikipedia.org/wiki/I%C2%B2C) (pronounced *I-squared-C*). In order to use this protocol, we need to enable the I<sup>2</sup>C interface on the RPi: go to the RPi Configuration from Preferences, then on the Interfaces tab for I<sup>2</sup>C click enable, then OK (you should have already done this from setup day).

It is not important to know all the details of how I<sup>2</sup>C works to use it, so this notebook contains just an abbreviated explanation.

All of the devices are synced together by the clock pin of the master device, denoted by SCL for **serial clock**; this clock signal is simply a square wave that repeats. Data from all devices are then sent over the data wire, denoted SDA for **serial data**. The data sent across this wire are synced up with the clock, where a high voltage on the data line indicates a 1 bit, and a low voltage indicates a 0 bit. 

Every subordinate device has an address that is assigned when the chip is manufactured; the master device is informed of the addresses by the programmer. The first data sent out by the master is the address of the device; all of the subordinate devices read the address to see if the master is communicating with them. The master can then send data to the appropriate subordinate device, and all other devices ignore these data. Once the master stops sending data, the subordinate device can send its own data to the master through the SDA wire. 

<img src="https://cdn.sparkfun.com/assets/6/4/7/1/e/51ae0000ce395f645d000000.png" width="90%">

This communication protocol may sound complicated, but it is standard, meaning that code already exists for us to use. The library we will use is called `smbus` for **System Management Bus**, and gives us very generic code to communicate with any device over I<sup>2</sup>C. This is the part where the [LIS3DH datasheet](https://cdn.sparkfun.com/assets/b/c/1/3/a/CD00274221.pdf) is useful, because we'll need to reference specific information from it in order to use the accelerometer. Only the most vital information will be summarized here.

The first step is to import the `smbus` library with 

`import smbus`. 

Once we've done that, we can create a new instance of the `smbus` object by typing 

`bus = smbus.SMBus(I2CPort)`,

where the port parameter is an integer representing the I<sup>2</sup>C bus; the RPi has 2 buses, and most often use the bus with index 1, so that is the number we use.

Note that with I<sup>2</sup>C, we don't need to specify the clock speed, because most sensors tend to just use the same speed of $100$kHz, which is considered normal speed. 

Run the code in the following cell to setup communication with the accelerometer.

In [ ]:
# load the smbus library to communicate with I2C devices
import smbus

# create a new smbus object on bus index 1 of the RPi
bus = smbus.SMBus(1)

As indicated earlier, every device has an address. On page 21 of the datasheet, the first sentence in the second paragraph states the address of the accelerometer as being  `0b001100` or `0b001101`. The reason it gives 2 options is because you could have 2 sensors on the same I<sup>2</sup>C bus that have the same address. They would both respond at the same time causing communication issues, so you can change the address to prevent problems. We are only using 1 sensor, so we don't have to worry about it; the default address will be `0b001101` because the last bit defaults to high. 

The binary number is useful to see when learning, but we often use hexadecimal to make writing these numbers more compact. Converting `0b001101` to hexadecimal gives `0x19` (note the `0x` prefix to indicate hexadecimal), which is the same value that is referenced in the datasheet.

Every I<sup>2</sup>C device has a set of registers that can either be written to or read by the master device. On page 26 of the datasheet, Table 17 includes a list of registers that one can play with, followed a description of their usage. Each provides different benefits, so feel free to explore and experiment with them once you're comfortable using the accelerometer. We only need a few of them to get basic functionality, so those will be described here.

Page 29 of the datasheet shows how the sensor transitions between various modes of operation. Just after the sensor is powered on, it goes into **power down mode**. As explained in Table 26, we can't read acceleration data in power down mode, so we need to go to the **normal mode**. Page 29 of the datasheet contains information about the first control register, which allows us to put the sensor into normal mode with a specified data rate configuration. Table 26 indicates that setting the first bits to 0111 corresponds to normal mode with 400kHz data rate. The next bit in the register enables us to select normal mode (set the bit to 0). The following 3 bits allow us to enable the x,y,z axes. So, we would like to set this register to `0b01110111` or in hex `0x77`.

>The SMBus library has a method called `bus.write_byte_data(address, register, data)` that allows us to write data to a device. The first parameter is the address of the device, the second is the register that we want to write to, and data is what we want to write. As you can see in the datasheet, the hexadecimal value of the first control register is `0x77`. This means the line of code we need to write to go into active mode is `bus.write_byte_data(0x19, 0x20, 0x77)`.

To quickly summarize what we did, we can't read acceleration data from the sensor until we go into normal mode, so we set the active bit in one of the control registers; this is the first line of code in the cell below. That's a lot of information for a single line of code, but it is outlining the process for how to make use of an I<sup>2</sup>C sensor.

In [ ]:
# set to active mode se we can read from the acceleration registers
bus.write_byte_data(0x19, 0x20, 0x77)

Now that the sensor is active, we can read the acceleration data. This sensor is a 3-axis accelerometer, meaning it measures acceleration in the $x$, $y$, and $z$ directions. Page 33 of the datasheet describes how the registers store the acceleration data: each axis has 12 bits of resolution, which are each stored in 2 registers. Because each register is 8 bits in size (1 byte), that leaves 4 extra unused bits that are always 0. If you look at the registers for the $x$ axis (`0x28` and `0x29`), the first register contains the least significant 8 bits, and the second contains the most significant 8 bits. We'll make use of bitwise operators to combine the data into a single number.

> The SMBus library includes a method to read a set of registers sequentially from the sensor `bus.read_i2c_block_data(address, register, numBytes)`, where the first parameter is the address of the device, the second is the first register to read, and the third is the number of bytes to read (must be >0). The method returns an array of bytes from each register, so that needs to be stored somewhere. The $x$ acceleration registers start at `0x28`.

In [ ]:
# read data from acceleration registers
# 2 bytes for each component
OUTX_REG = 0x28
xLSB = bus.read_i2c_block_data(0x19, OUTX_REG,   1)
xMSB = bus.read_i2c_block_data(0x19, OUTX_REG+1, 1)
print(format(xMSB[0],"08b"), format(xLSB[0],"08b") )

OUTY_REG = 0x2A
yLSB = bus.read_i2c_block_data(0x19, OUTY_REG,   1)
yMSB = bus.read_i2c_block_data(0x19, OUTY_REG+1, 1)
print(format(yMSB[0],"08b"), format(yLSB[0],"08b"))

OUTZ_REG = 0x2C
zLSB = bus.read_i2c_block_data(0x19, OUTZ_REG,   1)
zMSB = bus.read_i2c_block_data(0x19, OUTZ_REG+1, 1)
print(format(zMSB[0],"08b"), format(zLSB[0],"08b"))

Once we have the raw data from the registers, we can make use of the bitwise operators to make single numbers for each axis, then print out each one.

If you look on the top of the sensor, there is a printed label on the side indicating the direction of each axis, with $x$ to the right, $y$ towards you, and $z$ pointing upwards (depending on how you look at the sensor). Because Earth's gravity pulls on the sensor, we should expect to measure $9.81$m/s$^2$ on the $z$ axis and $0$ on the other axes when the sensor is flat. If you get a bunch of zeros, then you're not actually getting data from the accelerometer. There's probably an issue with your wiring somewhere or a code bug, so double check all of the setup.

In [ ]:
# combine x bits from its registers

# convert data from registers to single numbers
xRaw = (xMSB[0] << 4) + (xLSB[0] >> 4)
yRaw = (yMSB[0] << 4) + (yLSB[0] >> 4)
zRaw = (zMSB[0] << 4) + (zLSB[0] >> 4)

# display our data
print("xRaw", xRaw)
print("yRaw", yRaw)
print("zRaw", zRaw)

Let's take a look back in the datasheet to get an understanding of what we're looking at. Page 15 describes the data as being 2’s complement 162 bit numbers; this means we'll need to adjust each number when it goes negative. The resolution of each data register depends on the measurement range, which can be set to $\pm$2g (1g/1024), $\pm$4g (1g/512), or $\pm$8g (1g/256).

We next need to find what the full scale of the sensor is set to. Page 22 of the datasheet describes the data configuration register, which allows us to set the full scale of the accelerometer. Table 15 states that the last 2 bits, which set the full scale, default to `0b00`. Table 16 shows how to set the bits to change the measurement range, and `0b00` corresponds to 2g. If you wanted, you could change the full scale, but now we have the information we need to decode the data that we read from the sensor!

Let's first deal with the 2's complement formatting. The numbers returned from the previous cell were all positive integers, because that's how Python interprets the binary values. It doesn't know that the number is in 2's complement, so it assumes it's just positive. Even if the binary number is `0b1111 1111 1111` making -1, it's actually just going to interpret that as 4095. Run the code in the cell below to see the binary form of each number.

In [ ]:
# print numbers in binary so we can see the bits
print("xRaw", format(xRaw, "012b"), xRaw)
print("yRaw", format(yRaw, "012b"), yRaw)
print("zRaw", format(zRaw, "012b"), zRaw)

If any of your values start with a 1 bit, that means it's a negative value. In our code we'll need to check to see if the first bit is 1 or 0. If it's 1, then we need to force the number to be a negative value. Because the number is in 2's complement format, we can do this simply by subtracting 2<sup>12</sup> (or 4096) from the number. To understand why this works, think about the edge cases: 
* if we have `0b1111 1111 1111`, that represents -1 but it's interpreted as +4095 (or 2<sup>12</sup>-1). Subtracting 4096 from 4095 gives us -1 as we want. 
* if we have `0b1000 0000 0000` which represents -2048 but is interpreted as +2048. Again subtracting 4096 gives us the correct negative value. 

To quickly recap the fix for 2's complement numbers, we need to check if the first bit is 0. If so, then we subtract 2<sup>12</sup>=4096 from the number, which makes Python give us the right number. That's the first step to adjust the numbers, next we need to scale the values as described by the datasheet. Since we are using the 2g range, that gave us a resolution of 1g/1024, so if we simply divide each number by 1024 we will get the acceleration in g's. If you instead want m/s$^2$, you can then just multiply by $9.81 m/s^2$. Then we're done! If you run the code in the following cell, you should have roughly $0$ in the $x$ and $y$ directions and $9.81 m/s^2$ in the $z$ direction.

In [ ]:
# Fix for 2's complement numbers by checking what the value of the 12th digit is.
# We obtain this number by binary shift (>>) 11 times.
# If the number is negative (i.e. the remaining digit is 1), subtract 4096.
if xRaw >> 11 == 1: xRaw -= 2**12
if yRaw >> 11 == 1: yRaw -= 2**12
if zRaw >> 11 == 1: zRaw -= 2**12
    
# print numbers in binary so we can see the bits
print("xRaw", format(xRaw, "012b"), xRaw)
print("yRaw", format(yRaw, "012b"), yRaw)
print("zRaw", format(zRaw, "012b"), zRaw)

In [ ]:
# convert to acceleration
# divide by 1024 units (+/- 2g range, 1g per 1024 units)
# multiply by 9.81 for acceleration in m/s^2
toAcc = 9.81 / 1024

xAcc = xRaw * toAcc
yAcc = yRaw * toAcc
zAcc = zRaw * toAcc

# print the values in m/s^2
print("xAcc %+0.3f m/s^2"%xAcc)
print("yAcc %+0.3f m/s^2"%yAcc)
print("zAcc %+0.3f m/s^2"%zAcc)

Hopefully everything went well and you're getting values that make sense. You can try taking measurements with the accelerometer in different orientation to see what it gives you. If you later decide to change the full scale of the accelerometer, you also need to change the scaling value as described earlier. You should have enough information to be able to use the sensor in future applications and try to make use of the other features it offers, which will require you to read about them in the datasheet.